In [1]:
# ==================== Cell 1: Shared Cache Bootstrap ====================
"""
目標：建立統一快取機制，所有模型/資料集都存放在 AI_CACHE_ROOT
重要：這段程式碼會被複製到每本 notebook 的第一格
"""
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

[Cache] ../ai_warehouse/cache | GPU: True


In [2]:
# ===========================================
# Cell 2: Import Dependencies
# ===========================================

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextIteratorStreamer,
    BitsAndBytesConfig,
)
from threading import Thread
import time
from typing import List, Dict, Generator, Optional, Union
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

print("✓ Dependencies loaded")

c:\Users\b0979\miniconda3\envs\env-ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\b0979\miniconda3\envs\env-ai\lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


✓ Dependencies loaded


In [3]:
# ===========================================
# Cell 3: LLMAdapter Base Class Design
# ===========================================


class LLMAdapter:
    """
    Unified interface for different LLM backends.
    Supports: transformers, llama.cpp, ollama
    """

    def __init__(
        self,
        model_id: str,
        backend: str = "transformers",
        quantization: Optional[str] = None,  # "4bit", "8bit", None
        device_map: str = "auto",
        torch_dtype: str = "auto",
        **kwargs,
    ):

        self.model_id = model_id
        self.backend = backend
        self.quantization = quantization
        self.device_map = device_map
        self.torch_dtype = torch_dtype

        # Initialize backend
        if backend == "transformers":
            self._init_transformers(**kwargs)
        else:
            raise NotImplementedError(f"Backend {backend} not implemented yet")

    def _init_transformers(self, **kwargs):
        """Initialize Transformers backend with optional quantization"""
        print(f"🔄 Loading {self.model_id} (transformers backend)")

        # Setup quantization config
        quant_config = None
        if self.quantization == "4bit":
            quant_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
            )
            print("  └─ Using 4-bit quantization")
        elif self.quantization == "8bit":
            quant_config = BitsAndBytesConfig(load_in_8bit=True)
            print("  └─ Using 8-bit quantization")

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_id, use_fast=True, trust_remote_code=True
        )

        # Ensure pad token exists
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load model
        model_kwargs = {
            "device_map": self.device_map,
            "quantization_config": quant_config,
            "trust_remote_code": True,
            **kwargs,
        }

        # Handle torch_dtype
        if self.torch_dtype == "auto":
            model_kwargs["torch_dtype"] = (
                torch.float16 if torch.cuda.is_available() else torch.float32
            )
        elif self.torch_dtype:
            model_kwargs["torch_dtype"] = getattr(torch, self.torch_dtype)

        self.model = AutoModelForCausalLM.from_pretrained(self.model_id, **model_kwargs)

        print(f"✓ Model loaded on {self.model.device}")
        if hasattr(self.model, "get_memory_footprint"):
            print(f"  └─ Memory: {self.model.get_memory_footprint() / 1e9:.2f} GB")

    def generate(
        self,
        messages: List[Dict[str, str]],
        max_new_tokens: int = 256,
        temperature: float = 0.7,
        top_p: float = 0.9,
        do_sample: bool = True,
        stream: bool = False,
    ) -> Union[str, Generator[str, None, None]]:
        """
        Generate response from messages.

        Args:
            messages: List of {"role": "user/assistant/system", "content": "..."}
            stream: If True, return generator for streaming
        """

        if self.backend == "transformers":
            return self._generate_transformers(
                messages, max_new_tokens, temperature, top_p, do_sample, stream
            )
        else:
            raise NotImplementedError(f"Backend {self.backend} not implemented")


print("✓ LLMAdapter base class defined")

✓ LLMAdapter base class defined


In [4]:
# ===========================================
# Cell 4: Messages Template Processing
# ===========================================


def format_messages_for_model(messages: List[Dict[str, str]], tokenizer) -> str:
    """
    Convert messages to model-specific chat template.
    Falls back to simple format if chat_template not available.
    """

    # Try using the model's chat template first
    if hasattr(tokenizer, "chat_template") and tokenizer.chat_template:
        try:
            formatted = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            return formatted
        except Exception as e:
            print(f"⚠️  Chat template failed: {e}")

    # Fallback to simple format
    formatted_parts = []
    for msg in messages:
        role = msg["role"]
        content = msg["content"]

        if role == "system":
            formatted_parts.append(f"System: {content}")
        elif role == "user":
            formatted_parts.append(f"User: {content}")
        elif role == "assistant":
            formatted_parts.append(f"Assistant: {content}")

    # Add generation prompt
    formatted_parts.append("Assistant:")

    return "\n".join(formatted_parts)


# Test message formatting
test_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "什麼是 RAG？請用繁體中文回答。"},
]

print("✓ Message formatting function defined")
print("Sample messages:", test_messages)

✓ Message formatting function defined
Sample messages: [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '什麼是 RAG？請用繁體中文回答。'}]


In [5]:
# ===========================================
# Cell 5: Transformers Backend Implementation
# ===========================================


def _generate_transformers(
    self, messages, max_new_tokens, temperature, top_p, do_sample, stream
):
    """Transformers backend generation with streaming support"""

    # Format messages
    prompt = format_messages_for_model(messages, self.tokenizer)

    # Tokenize
    inputs = self.tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096,  # Prevent overly long prompts
    ).to(self.model.device)

    # Generation parameters
    gen_kwargs = {
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "do_sample": do_sample,
        "pad_token_id": self.tokenizer.pad_token_id,
        "eos_token_id": self.tokenizer.eos_token_id,
    }

    if stream:
        # Streaming generation
        streamer = TextIteratorStreamer(
            self.tokenizer, timeout=30.0, skip_prompt=True, skip_special_tokens=True
        )
        gen_kwargs["streamer"] = streamer

        # Start generation in separate thread
        thread = Thread(target=self.model.generate, kwargs={**inputs, **gen_kwargs})
        thread.start()

        # Yield tokens as they come
        for token in streamer:
            yield token

    else:
        # Non-streaming generation
        with torch.no_grad():
            outputs = self.model.generate(**inputs, **gen_kwargs)

        # Decode response (skip input tokens)
        response = self.tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[1] :], skip_special_tokens=True
        )
        return response.strip()


# Monkey patch the method to LLMAdapter
LLMAdapter._generate_transformers = _generate_transformers

print("✓ Transformers backend implementation added")

✓ Transformers backend implementation added


In [6]:
# ===========================================
# Cell 6: Low-VRAM Model Loading Examples
# ===========================================

# Model recommendations by VRAM
VRAM_RECOMMENDATIONS = {
    "4GB": ["microsoft/DialoGPT-small", "distilgpt2"],
    "8GB": ["Qwen/Qwen2.5-3B-Instruct", "microsoft/DialoGPT-medium"],
    "12GB": ["Qwen/Qwen2.5-7B-Instruct", "meta-llama/Llama-3.2-3B-Instruct"],
    "16GB+": ["Qwen/Qwen2.5-14B-Instruct", "meta-llama/Llama-3.1-8B-Instruct"],
}


def suggest_model_for_vram():
    """Suggest appropriate model based on available VRAM"""
    if not torch.cuda.is_available():
        return "microsoft/DialoGPT-small"  # CPU fallback

    try:
        vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"📊 Available VRAM: {vram_gb:.1f} GB")

        if vram_gb >= 16:
            return "Qwen/Qwen2.5-7B-Instruct"  # Will use 4-bit
        elif vram_gb >= 12:
            return "Qwen/Qwen2.5-7B-Instruct"  # Definitely need 4-bit
        elif vram_gb >= 8:
            return "Qwen/Qwen2.5-3B-Instruct"
        else:
            return "microsoft/DialoGPT-medium"

    except Exception:
        return "microsoft/DialoGPT-small"


recommended_model = suggest_model_for_vram()
print(f"💡 Recommended model: {recommended_model}")

# Low-VRAM loading strategies
print("\n🔧 Low-VRAM strategies:")
print("1. 4-bit quantization: quantization='4bit'")
print("2. CPU offloading: device_map='auto'")
print("3. Smaller models: 3B/7B instead of 14B+")
print("4. Gradient checkpointing (for training)")

📊 Available VRAM: 17.1 GB
💡 Recommended model: Qwen/Qwen2.5-7B-Instruct

🔧 Low-VRAM strategies:
1. 4-bit quantization: quantization='4bit'
2. CPU offloading: device_map='auto'
3. Smaller models: 3B/7B instead of 14B+
4. Gradient checkpointing (for training)


In [9]:
# ===========================================
# Cell 7: Smoke Test - Initialize and Test
# ===========================================

print("🧪 Starting Smoke Test...")

# Use a small model for testing (adjust if needed)
test_model = "Qwen/Qwen2.5-7B-Instruct"

try:
    # Initialize adapter
    adapter = LLMAdapter(
        model_id=test_model,
        backend="transformers",
        quantization="8-bit",  # No quantization for small model
        device_map="auto",
    )

    # Test messages
    test_messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "請介紹甚麼是RAG 它的應用有哪一些"},
    ]

    print("\n📝 Testing non-streaming generation...")
    start_time = time.time()
    response = adapter.generate(
        messages=test_messages, max_new_tokens=50, temperature=0.7, stream=False
    )
    end_time = time.time()

    print(f"✓ Response: {response}")
    print(f"⏱️  Time: {end_time - start_time:.2f}s")

    print("\n🌊 Testing streaming generation...")
    stream_response = ""
    for token in adapter.generate(
        messages=test_messages, max_new_tokens=30, temperature=0.7, stream=True
    ):
        print(token, end="", flush=True)
        stream_response += token

    print(f"\n✓ Streaming complete. Total: {len(stream_response)} chars")

    print("\n🎉 Smoke test PASSED!")

except Exception as e:
    print(f"❌ Smoke test FAILED: {e}")
    print("💡 Try adjusting the model or check your environment")

🧪 Starting Smoke Test...
🔄 Loading Qwen/Qwen2.5-7B-Instruct (transformers backend)


Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.07s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


✓ Model loaded on cuda:0
  └─ Memory: 15.23 GB

📝 Testing non-streaming generation...
✓ Response: <generator object _generate_transformers at 0x00000195383EE180>
⏱️  Time: 0.01s

🌊 Testing streaming generation...
RAG 是 Retrieval-Augmented Generation（检索增强生成）的縮寫，是一種利用語音模型進行生成任務的方法。它結合
✓ Streaming complete. Total: 70 chars

🎉 Smoke test PASSED!


In [10]:
# ===========================================
# Cell 8: Performance Monitoring
# ===========================================


def get_model_info(adapter):
    """Get basic model information and memory usage"""
    info = {
        "model_id": adapter.model_id,
        "backend": adapter.backend,
        "quantization": adapter.quantization,
        "device": str(adapter.model.device) if hasattr(adapter, "model") else "unknown",
    }

    # Memory info
    if torch.cuda.is_available() and hasattr(adapter, "model"):
        try:
            info["vram_allocated"] = f"{torch.cuda.memory_allocated() / 1e9:.2f} GB"
            info["vram_cached"] = f"{torch.cuda.memory_reserved() / 1e9:.2f} GB"
        except:
            pass

    # Model parameters
    if hasattr(adapter, "model"):
        try:
            total_params = sum(p.numel() for p in adapter.model.parameters())
            info["parameters"] = f"{total_params / 1e6:.1f}M"
        except:
            pass

    return info


# Test performance monitoring (if adapter was created successfully)
try:
    model_info = get_model_info(adapter)
    print("📊 Model Information:")
    for key, value in model_info.items():
        print(f"  {key}: {value}")
except NameError:
    print("⚠️  No adapter to analyze (smoke test may have failed)")

📊 Model Information:
  model_id: Qwen/Qwen2.5-7B-Instruct
  backend: transformers
  quantization: 8-bit
  device: cuda:0
  vram_allocated: 12.29 GB
  vram_cached: 13.58 GB
  parameters: 7615.6M


In [11]:
# ===========================================
# Cell 9: Key Parameters and Configuration
# ===========================================

print("🎛️  Key Parameters for LLMAdapter:")
print()
print("📦 Model Selection:")
print("  • model_id: HuggingFace model identifier")
print("  • quantization: '4bit', '8bit', or None")
print("  • device_map: 'auto', 'cpu', or specific device")
print()
print("🔧 Generation Parameters:")
print("  • max_new_tokens: Output length limit (default: 256)")
print("  • temperature: Randomness (0.1=conservative, 1.0=creative)")
print("  • top_p: Nucleus sampling (0.9 recommended)")
print("  • stream: True for real-time output")
print()
print("💾 Memory Optimization:")
print("  • 4-bit quantization: ~75% memory reduction")
print("  • device_map='auto': Automatic CPU/GPU distribution")
print("  • torch_dtype='float16': Half precision (GPU only)")

# Configuration examples for different scenarios
configs = {
    "high_quality": {
        "quantization": None,
        "torch_dtype": "float16",
        "temperature": 0.3,
        "top_p": 0.9,
    },
    "low_vram": {
        "quantization": "4bit",
        "device_map": "auto",
        "temperature": 0.7,
        "max_new_tokens": 128,
    },
    "creative_writing": {
        "quantization": "8bit",
        "temperature": 0.8,
        "top_p": 0.95,
        "max_new_tokens": 512,
    },
}

print("\n📋 Preset Configurations:")
for name, config in configs.items():
    print(f"  {name}: {config}")

🎛️  Key Parameters for LLMAdapter:

📦 Model Selection:
  • model_id: HuggingFace model identifier
  • quantization: '4bit', '8bit', or None
  • device_map: 'auto', 'cpu', or specific device

🔧 Generation Parameters:
  • max_new_tokens: Output length limit (default: 256)
  • temperature: Randomness (0.1=conservative, 1.0=creative)
  • top_p: Nucleus sampling (0.9 recommended)
  • stream: True for real-time output

💾 Memory Optimization:
  • 4-bit quantization: ~75% memory reduction
  • device_map='auto': Automatic CPU/GPU distribution
  • torch_dtype='float16': Half precision (GPU only)

📋 Preset Configurations:
  high_quality: {'quantization': None, 'torch_dtype': 'float16', 'temperature': 0.3, 'top_p': 0.9}
  low_vram: {'quantization': '4bit', 'device_map': 'auto', 'temperature': 0.7, 'max_new_tokens': 128}
  creative_writing: {'quantization': '8bit', 'temperature': 0.8, 'top_p': 0.95, 'max_new_tokens': 512}


In [12]:
# ===========================================
# Cell 10: What We Built & Next Steps
# ===========================================

print("🏗️  What We Built in nb02:")
print("✓ LLMAdapter 統一介面設計")
print("✓ Transformers 後端實現 (with 4/8-bit quantization)")
print("✓ Messages 格式處理與模板系統")
print("✓ 串流輸出支援 (TextIteratorStreamer)")
print("✓ 低 VRAM 友善設計 (device_map, quantization)")
print("✓ 效能監控與記憶體追蹤")

print("\n⚠️  Key Pitfalls:")
print("• 模型快取路徑設定錯誤會重複下載")
print("• 4-bit 量化可能影響輸出品質")
print("• 串流輸出需要額外的 threading 處理")
print("• Chat template 不相容時需要 fallback")

print("\n🎯 Next Steps (nb03-nb08):")
print("• nb03: llama.cpp 後端 (GGUF 支援)")
print("• nb04: Ollama HTTP 後端")
print("• nb05: 更精細的 chat template 處理")
print("• nb06: 延遲、throughput 基準測試")
print("• nb07: JSONL 資料格式與驗證")
print("• nb08: 輸入/輸出安全檢查")

print("\n🔄 Reproducibility Tips:")
print("• 設定 AI_CACHE_ROOT 環境變數避免重複下載")
print("• 使用 torch.manual_seed() 保證可重現輸出")
print("• 記錄 VRAM 使用量以便除錯")
print("• 保存 model_info 以便版本追蹤")

print("\n✅ Ready for Stage 1 integration!")

🏗️  What We Built in nb02:
✓ LLMAdapter 統一介面設計
✓ Transformers 後端實現 (with 4/8-bit quantization)
✓ Messages 格式處理與模板系統
✓ 串流輸出支援 (TextIteratorStreamer)
✓ 低 VRAM 友善設計 (device_map, quantization)
✓ 效能監控與記憶體追蹤

⚠️  Key Pitfalls:
• 模型快取路徑設定錯誤會重複下載
• 4-bit 量化可能影響輸出品質
• 串流輸出需要額外的 threading 處理
• Chat template 不相容時需要 fallback

🎯 Next Steps (nb03-nb08):
• nb03: llama.cpp 後端 (GGUF 支援)
• nb04: Ollama HTTP 後端
• nb05: 更精細的 chat template 處理
• nb06: 延遲、throughput 基準測試
• nb07: JSONL 資料格式與驗證
• nb08: 輸入/輸出安全檢查

🔄 Reproducibility Tips:
• 設定 AI_CACHE_ROOT 環境變數避免重複下載
• 使用 torch.manual_seed() 保證可重現輸出
• 記錄 VRAM 使用量以便除錯
• 保存 model_info 以便版本追蹤

✅ Ready for Stage 1 integration!


In [ ]:
# ===========================================
# Cell 11: Save Adapter to Shared Utils (Optional)
# ===========================================

# Create shared_utils directory structure
import os

os.makedirs("../shared_utils/adapters", exist_ok=True)

# Save the LLMAdapter class for reuse
adapter_code = '''
"""
LLMAdapter for ragent-text-lab
Unified interface for transformers/llama.cpp/ollama backends
"""

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextIteratorStreamer,
    BitsAndBytesConfig
)
from threading import Thread
import torch
from typing import List, Dict, Generator, Optional, Union

class LLMAdapter:
    """Unified LLM interface supporting multiple backends"""

    def __init__(self, model_id: str, backend: str = "transformers", **kwargs):
        self.model_id = model_id
        self.backend = backend

        if backend == "transformers":
            self._init_transformers(**kwargs)
        else:
            raise NotImplementedError(f"Backend {backend} not implemented yet")

    def _init_transformers(self, quantization=None, device_map="auto", torch_dtype="auto", **kwargs):
        # Implementation from notebook...
        pass

    def generate(self, messages: List[Dict[str, str]], **kwargs):
        # Implementation from notebook...
        pass
'''

with open("../shared_utils/adapters/llm_adapter.py", "w") as f:
    f.write(adapter_code)

print("💾 LLMAdapter saved to shared_utils/adapters/llm_adapter.py")
print(
    "📝 Ready to import in future notebooks: from shared_utils.adapters import LLMAdapter"
)